In [2]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

2024-07-27 12:06:13.815864: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-27 12:06:13.843045: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 12:06:13.843079: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 12:06:13.843735: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-27 12:06:13.847847: I tensorflow/core/platform/cpu_feature_guar

In [3]:
import csv

In [4]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [5]:
images_path = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Images"
label_path = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Airplanes_Annotations"

In [6]:
#there are 700 images with name starting with airplane, and 33 with numbers
img_bb = {}

In [7]:
for label in os.listdir(label_path):
    if label.split("_")[0] == "airplane":
        try:
            file = pd.read_csv(label_path+"/"+label)
            # print("label: ", label)
            # print("len of file: ", len(file))
            labels_for_image = []
            for row in file.iterrows():
                # for k in row:
                #     print(row[k][0].split(" "))
                bb_each = list(row[1])[0].split(" ")
                for j in range(len(bb_each)):
                    bb_each[j] = eval(bb_each[j])

                labels_for_image.append(bb_each)
            img_bb[label.split(".")[0]] = labels_for_image
        except Exception as e:
            print(e)
            print("Error in :", label)

In [8]:
print("Total images for training: ", len(img_bb))

Total images for training:  700


In [9]:
for i in img_bb:
    print("Image name: ", i)
    print("Number of labels: ", len(img_bb[i]))
    print("Labels: ", img_bb[i])

Image name:  airplane_201
Number of labels:  26
Labels:  [[22, 78, 31, 90], [29, 69, 48, 81], [44, 65, 59, 76], [58, 57, 68, 70], [122, 16, 138, 26], [134, 9, 146, 21], [93, 35, 102, 46], [79, 41, 93, 56], [69, 48, 79, 59], [27, 110, 39, 121], [38, 102, 48, 111], [48, 97, 62, 105], [61, 89, 76, 97], [73, 82, 86, 91], [87, 75, 96, 84], [106, 69, 118, 79], [118, 53, 133, 65], [133, 47, 145, 58], [144, 39, 154, 48], [116, 238, 127, 253], [126, 232, 143, 248], [136, 225, 152, 241], [154, 216, 164, 226], [165, 209, 179, 219], [166, 220, 174, 230], [152, 229, 165, 237]]
Image name:  airplane_407
Number of labels:  3
Labels:  [[30, 90, 78, 131], [143, 6, 190, 46], [200, 180, 253, 236]]
Image name:  airplane_343
Number of labels:  2
Labels:  [[122, 153, 197, 247], [79, 5, 147, 98]]
Image name:  airplane_520
Number of labels:  4
Labels:  [[63, 116, 92, 137], [107, 91, 132, 115], [140, 128, 165, 156], [176, 230, 202, 254]]
Image name:  airplane_296
Number of labels:  2
Labels:  [[5, 89, 93, 193]

In [10]:
ele1n = ""
elebb1 = []

In [11]:
def gt(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [15]:
#we pass each image for selective search and calculate iou for each bb output of ss out of 2000 proposals
cnt = 0
for img in os.listdir(images_path):
    if cnt == 1:
        break
    try:
        if img.split("_")[0] == "airplane":
            image = cv2.imread(os.path.join(images_path, img))
            ss.setBaseImage(image)
            ss.switchToSelectiveSearchFast(base_k=10)
            ssresults = ss.process()
            imout = image.copy()
            print("image name: ", img)
            print("len of ssresults: ", len(ssresults))
            print("going to show first 5 elements of ssresults: ")
            ele1n = os.path.join(images_path, img)
            print("name: ", ele1n)
            print("bb: ")
            for i in ssresults:
                x1, y1, w, h = i
                bb2 = {"x1":x1, "y1":y1, "x2":x1+w, "y2":y1+h}
                for i in bb2:
                    print("key:value", bb2[i])
                print("hihihhihihihhihi")
                print("len(img_bb):", len(img_bb))
                img = img.split(".")[0]
                print("img: ", img)
                if img in img_bb:
                    print("yes its present")
                print("len(img_bb):", len(img_bb[img]))
                for j in img_bb[img]:
                    print("we are checking the labels of image : ", img)
                    x, y, xn, yn = j
                    bb1 = {"x1":x, "y1":y, "x2":xn, "y2":yn}
                    iou = gt(bb1, bb2)
                    print("sjdjkskdjsjdjfkjksjdkfjkjskjjsjfkjkfjkjsfjksdjf: ", iou)
                    if iou > 0.7:
                        cv2.rectangle(image, (bb1["x1"], bb1["y1"]), (bb1["x2"], bb1["y2"]), (255, 0, 0), 2)
                        cv2.rectangle(image, (bb2["x1"], bb2["y1"]), (bb2["x2"], bb2["y2"]), (0, 255, 0), 1)
            cv2.imshow(img, image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        cnt += 1
    except Exception as e:
        print(e)
        print("Error in image: ", img)
        continue

image name:  airplane_333.jpg
len of ssresults:  1024
going to show first 5 elements of ssresults: 
name:  /home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Images/airplane_333.jpg
bb: 
key:value 188
key:value 44
key:value 224
key:value 76
hihihhihihihhihi
len(img_bb): 700
img:  airplane_333
yes its present
len(img_bb): 3
we are checking the labels of image :  airplane_333
sjdjkskdjsjdjfkjksjdkfjkjskjjsjfkjkfjkjsfjksdjf:  0.0
we are checking the labels of image :  airplane_333
sjdjkskdjsjdjfkjksjdkfjkjskjjsjfkjkfjkjsfjksdjf:  0.0
we are checking the labels of image :  airplane_333
sjdjkskdjsjdjfkjksjdkfjkjskjjsjfkjkfjkjsfjksdjf:  0.07425742574257425
key:value 50
key:value 158
key:value 67
key:value 195
hihihhihihihhihi
len(img_bb): 700
img:  airplane_333
yes its present
len(img_bb): 3
we are checking the labels of image :  airplane_333
sjdjkskdjsjdjfkjksjdkfjkjskjjsjfkjkfjkjsfjksdjf:  0.3371394230769231
we are checking the labels of image :  airplane_333


In [13]:
i = cv2.imread("/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Images/airplane_343.jpg")
cv2.imshow("image", i)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
x1, y1, x2, y2 = ele1bb[0]
print(x1, y1, x1+x2, y1+y2)

NameError: name 'ele1bb' is not defined

In [ ]:
re1 = [(x1, y1), (x1+ x2, y1+y2)]

In [ ]:
jk = cv2.imread(ele1n)
cv2.rectangle(jk, re1[0], re1[1], (0, 0, 255), 3)
cv2.imshow("image", jk)
cv2.waitKey(0)
cv2.destroyAllWindows()